# Scraping og API-kald i Python

## Program

- Opfølgning fra sidste undervisningsgang
- At tilgå internettet med Python ("requests" i Python)
- API-kald i Python (eksempler fra Reddit Pushshift API og Twitter API)
- Scraping af HTML med `BeautifulSoup`
- Introduktion til crawlers med `scrapy`

# Opfølgning fra sidste undervisningsgang

![dns](../img/dns.png)

## At tilgå internettet i Python

To skridt involveret i at samle data fra internettet:

**1. Send (http) request (GET eller POST)**

**2. Behandl indhold**
- For scraping: Kildekode (HTML)
- For API'er: Afhænger af API (ofte i JSON format)
    
Ved API'er bruger man længst tid på 1 (hvordan virker API'en, og hvordan formulerer jeg den rigtige forespørgsel?)

Ved scraping bruger man længst tid på 2 (hvordan sorterer jeg i hjemmesidens kildekode?)

## HTTP requests

<img src="https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/How_the_Web_works/simple-client-server.png" style="width:50.0%" />

**Request**
- Det, som vi afsender
- GET requests: Typisk brugt for at *anmode* om information (API og browser)
- POST requests: Typisk brugt for at *tilføje* noget
    - Grundet begrænsninger ved GET requests, bruges POST requests også nogen gange til at hente information fra API'er

**Response**
- Det, som returneres
- Data af en eller anden form - afhængiger af modtagende server

## Statuskoder

En HTTP request returnerer altid en statuskode.

- Statuskode der starter med 2 eller 3: Request successful
    - 200 = "OK"
- Statuskode der starter med 4: Request har fejlet ("client-side", fx 404).
- Statuskode der starter med 5: Request har fejlet (server-side)

## Requests med `requests`

`requests` indeholder funktioner til at sende HTTP requests.

En browser sender en GET request, når den skal "hente" en hjemmeside:

In [2]:
import requests

r = requests.get("https://www.aau.dk")

`r` er nu en "response" class med forskellige attributter; fx `.status_code`:

In [7]:
r.status_code

200

`.content` giver indholdet (her HTML kildekode):

In [12]:
r.content[0:1000]

b'<!DOCTYPE html><html><head><link rel="stylesheet" data-href="https://fonts.googleapis.com/css2?family=Barlow:ital,wght@0,100;0,200;0,300;0,400;0,500;0,600;0,700;0,800;0,900;1,100;1,200;1,300;1,400;1,500;1,600;1,700;1,800;1,900&amp;family=Montserrat:ital,wght@0,100;0,200;0,300;0,400;0,500;0,600;0,700;0,800;0,900;1,100;1,200;1,300;1,400;1,500;1,600;1,700;1,800;1,900&amp;family=Space+Mono:ital,wght@0,400;0,700;1,400&amp;display=swap"/><link rel="preconnect" href="https://fonts.gstatic.com" crossorigin /><meta name="viewport" content="width=device-width"/><meta charSet="utf-8"/><title>AAU - Viden for verden - Aalborg Universitet</title><meta name="description" content="Aalborg Universitet - Problem- og projektbaseret forskning og uddannelse, der i samspil mellem AAU og omverdenen skaber viden, der forandrer verden."/><meta name="robots" content="follow, index"/><link rel="canonical" href="https://www.aau.dk/"/><meta property="og:url" content="https://www.aau.dk/"/><meta property="og:type

## API-kald i Python

API'er er forskellige men indeholder typisk de samme delkomponenter:

**Request:** Brug af API involverer at sende HTTP request (GET eller POST).

**Endpoint:** API'er består typisk af flere *endpoints*. Disse er blot URL'er.

**Parametre:** Parametre er de *argumetnter*, som endpointet accepterer. Via disse formuleres, hvad der efterspørges af API'en. 

**Autentificeirng:** Mange API'er kræver autentificering. Dette kan være HTTPS autentificering (brugernavn og kodeord) eller autentificering via *tokens*. Tokens er unikke nøgler, der identificerer, hvem eller hvad der laver request/henvendelse via API'en.

## API-kald i Python (fiktivt eksempel)

```python
endpoint = "https://api.database.com/search" # hvilken URL har API'en endpoint?

authentication = ("myuser123", "!mypass123") # autentificering - her via brugernavn og adgangskode

parameters = {"date_from": "2022-07-10", # parametre - hvilke oplysninger skal hentes
              "date_to": "2022-08-10",
              "q": "popsicles"
             }

r = requests.get(endpoint, auth = authentication, params = parameters) # send get request (bemærk at "auth" og "params" er argumenter til requests.get())
```

## (Kort) Introduktion til HTML

Når man arbejder med "rå" web scraping, er materialet man får tilbage i form af rå kildekode.

Det ville være meget besværligt at sortere i rå kildekode, som det ser ud. 

HTML har en struktur, som kan "udnyttes" til at filtrere unødvendig information fra.

## (Kort) Introduktion til HTML

**Rå HTML**

```
    <html>
        <body>
            <div id="convo1">
                <p class="kenobi">Hello There!</p>
            </div>
            <div id="convo2">
                <p class="grievous">General Kenobi!</p>
            </div>
            <div id="convo3">
                <p class="kenobi">So Uncivilized!</p>
            </div>
        </body>
    </html>
```    

**Rendered (som vist på hjemmeside)**

```
Hello There!

General Kenobi!

So Uncivilized!
```

*Hvilken datastruktur minder rå HTML jer om?*

## (Kort) Introduktion til HTML - Tags

HTML står for "Hyper-Text Markup Language". Det bruges til at strukturere indhold på hjemmesider. 

HTML er opbygget af "tags" angivet med `<>` og `</>`. Disse fortæller hvilken type indhold, der er tale om. `<p>` er for eksempel "paragraph" (tekstafnit).
- Typiske tags: `h1` for overskrifter (og `h2`, `h3` osv.), `a` for links og `div` for e "division" eller adskillelse.

HTML har en træ-lignende struktur. Tags befinder sig inde i andre tags. 
- "Siblings"/"Søskende": Tags på samme niveau 
- "Children"/"Børn": Tags under/inden i et andet tag
- "Parents"/"Forældre": Tags over/uden for et andet tag

## (Kort) Introduktion til HTML - Attributes

HTML bruger "attributes" til både at differentiere mellem den samme type tag og til at tilføje yderligere information til et tag.

Opbyggeren af en hjemmeside kan selv navngive attributes, men visse standarder går igen:
- `id` attribute: Giver tags unikt id (ideelt set) 
- `class` attribute: Differentiere mellem forskellige tags af samme type - fx for at give forskellig styling/formatering
- `href` attribute: Indeholder URL som hyperlink refererer til (typisk en del af et `a` tag)

## (Kort) Introduktion til HTML

*Uden at søge på tekstindholdet, hvordan kan vi så udlede teksten "General Kenobi" af nedenstående HTML-kode?*

```
    <html>
        <body>
            <div id="convo1">
                <p class="kenobi">Hello There!</p>
            </div>
            <div id="convo2">
                <p class="grievous">General Kenobi!</p>
            </div>
            <div id="convo3">
                <p class="kenobi">So Uncivilized!</p>
            </div>
        </body>
    </html>
``` 

## Håndtering af HTML med `BeautifulSoup`

Pakken `BeautifulSoup` gør det nemmere at håndtere og navigere i HTML kode.

Pakkens funktioner er bygget op omkring et `soup` objekt.

Pakken fungerer ved at konvertere kildekode/HTML fra en hjemmeside (en string) til et `soup` objekt.

Man kan bruges HTML tags og attribute til at navigere i et `soup` objekt - blandt andet med metoderne `.find()` og `.find_all()`

## Håndtering af HTML med `BeautifulSoup`

In [14]:
### eksempel

## Introduktion til crawling

"Crawlers" eller "spiders" refererer typisk til programmer eller bots, der er bygget til at bevæge sig rundt på flere hjemmesider.

**En crawler består typisk af følgende:**
- Startbetingelser: Hvor skal crawleren starte?
- Parsing funktioner: Hvad skal crawleren gøre? (typisk en eller flere web scraping funktioner)
- Undtagelser: Hvad skal crawleren undgå?
- Slutbetingelser: Hvornår skal crawleren stoppe?

Grundet internettets opbygning, kan crawlere, der går på tværs af hjemmeider, være meget vanskelige at sætte op.

## Introduktion til crawling

*Crawlers bevæger sig altid mod overfladen!*

<img src="../img/web_sea.png" style="width:90.0% "/>


## Introduktion til crawling

En crawler kan sagtens sættes op med basis Python kommandoer:
- Definere scraping-funktion (`def`)
- Brug `requests` og `BeautifulSoup` til at skaffe og behandle hjemmesideindhold
- Gentag scraping, så længe der er flere links (fx med et `while` loop) 

En crawler kræver dog en del fejlhåndtering:
- Hvad skal der ske, hvis der ikke er flere links?
- Hvad skal der ske, hvis request fejler?
- Hvad skal der ske, hvis hjemmeside ikke indeholder det indhold, som scraper leder efter?
- Osv.

Derfor kan det ofte svare sig at bruge pakker til at bygge crawlers ud fra (fx `scrapy`).

## Introduktion til crawling - Advarsel!

*HUSK!:*

<img src="https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/How_the_Web_works/simple-client-server.png" style="width:50.0%" />

**Dette sker for *hver* henvendelse!**
- Scraping af enkelt hjemmeside: én request én gang
- Crawler: ??? requests inden for ??? tid

En crawler *skal* have forsinkelser indbygget mellem requests - ellers kan det betragtes som et angreb på server (robots.txt vil også nogen gange specificere et "crawl-delay")

## Introduktion til crawling

En simpel måde at skabe forsinkelser er fx med `time.sleep()`

In [23]:
from datetime import datetime
import time

start = datetime.now()

time.sleep(5)

end = datetime.now()

print(f"Jeg ventede i {(end-start).seconds} sekunder!")

Jeg ventede i 5 sekunder!


Scrapere fra `scrapy` vil dog have forsinkelse indbygget.